In [96]:
import os
import time
import copy
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import pytorch_lightning as pl

import torchvision
from torchvision import datasets, models, transforms

DATA_PATH = '../../datasets/crack-detection'

In [97]:
print(torch.version)
print(f'CUDA {torch.version.cuda}')
print(f'cuDNN {torch.backends.cudnn.version()}')

<module 'torch.version' from 'D:\\conda\\envs\\pytorch\\lib\\site-packages\\torch\\version.py'>
CUDA 10.2
cuDNN 7604


In [98]:
negative_count = len(os.listdir(DATA_PATH + '/Negative/'))
positive_count = len(os.listdir(DATA_PATH + '/Positive/'))

print(f'There are {negative_count} negative and {positive_count} positive samples.')

There are 20000 negative and 20000 positive samples.


In [99]:
class CrackDetectionModel(pl.LightningModule):
    def __init__(self):
        super().__init__()

        model = models.vgg16(pretrained=True)
        
        for param in model.parameters():
            param.requires_grad = False
        
        num_classes = 2
        model.classifier[6] = nn.Linear(4096, num_classes)
        self.model = model
    
    def forward(self, x):
        x = self.model(x)
        return x

    def prepare_data(self):
        image_folder = datasets.ImageFolder(DATA_PATH)

        train_size = int(0.5 * len(image_folder))
        valid_size = int(0.3 * len(image_folder))
        test_size  = len(image_folder) - (train_size + valid_size)
        train_ds, val_ds, test_ds = torch.utils.data.random_split(image_folder, [train_size, valid_size, test_size])
        
        print(f'There are {len(train_ds)} train., {len(val_ds)} valid. and {len(test_ds)} test samples.')

        train_ds.dataset.transform = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        val_ds.dataset.transform =  transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        test_ds.dataset.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        self.train_dataset = train_ds
        self.val_dataset   = val_ds
        self.test_dataset  = test_ds

    def train_dataloader(self):
        torch.utils.data.DataLoader(self.train_dataset, batch_size=32)

    def val_dataloader(self):
        torch.utils.data.DataLoader(self.val_dataset, batch_size=32)

    def test_dataloader(self):
        torch.utils.data.DataLoader(self.test_dataset, batch_size=32)

    def configure_optimizers(self):
        optimizer = optim.SGD(self.model.parameters(), lr=0.001, momentum=0.9)
        scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
        return optimizer, scheduler

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_hat = self.forward(x)
        return {'loss': F.cross_entropy(y_hat, y)}

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_hat = self.forward(x)
        return {'val_loss': F.cross_entropy(y_hat, y)}

    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        y_hat = self.forward(x)
        return {'test_loss': F.cross_entropy(y_hat, y)}

In [100]:
model = CrackDetectionModel()

trainer = pl.Trainer()
trainer.fit(model)

GPU available: True, used: False
No environment variable for node rank defined. Set as 0.

   | Name               | Type              | Params
-----------------------------------------------------
0  | model              | VGG               | 134 M 
1  | model.features     | Sequential        | 14 M  
2  | model.features.0   | Conv2d            | 1 K   
3  | model.features.1   | ReLU              | 0     
4  | model.features.2   | Conv2d            | 36 K  
5  | model.features.3   | ReLU              | 0     
6  | model.features.4   | MaxPool2d         | 0     
7  | model.features.5   | Conv2d            | 73 K  
8  | model.features.6   | ReLU              | 0     
9  | model.features.7   | Conv2d            | 147 K 
10 | model.features.8   | ReLU              | 0     
11 | model.features.9   | MaxPool2d         | 0     
12 | model.features.10  | Conv2d            | 295 K 
13 | model.features.11  | ReLU              | 0     
14 | model.features.12  | Conv2d            | 590 K 
15 | mo

TypeError: 'NoneType' object is not iterable